# Regularized Methods

- Feature Scaling
- Test/Train split
- Ridge, LASSO, Elastic Net Regression methods

---

In a regular linear scenario, we start with a regular linear function.

$$ \hat y = b + ax_0$$



The mean square error of these predictions would be given by:

$$RSS(a, b) = \sum_{i = 1}^n(y_i -  (ax_i + b))^2$$

From this basic $MSE$ formulation, we can introduce some Regularized methods that add a *regularization term*  to the $MSE$.  We will look at three methods that offer slight variations on this term.

### Feature Scaling

To use these methods, we want to scale our data.  Many Machine Learning algorithms don't do well with data operating on very different scales.  Using the `MinMaxScaler` normalizes the data and brings the values between 0 and 1. The `StandardScaler` method is less sensitive to wide ranges of values. We will use both on our Ames housing data.  To begin, we need to select the numeric columns from the DataFrame so we can transform them only.

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
#get our data and select the numer
ames = pd.read_csv('data/ames_housing.csv')
y = ames['SalePrice']
ames = ames.drop('SalePrice', axis = 1)

In [3]:
ames_numeric = ames.select_dtypes(include = 'int64')
ames_numeric.head()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,8450,7,5,2003,2003,706,0,150,...,548,0,61,0,0,0,0,0,2,2008
1,2,20,9600,6,8,1976,1976,978,0,284,...,460,298,0,0,0,0,0,0,5,2007
2,3,60,11250,7,5,2001,2002,486,0,434,...,608,0,42,0,0,0,0,0,9,2008
3,4,70,9550,7,5,1915,1970,216,0,540,...,642,0,35,272,0,0,0,0,2,2006
4,5,60,14260,8,5,2000,2000,655,0,490,...,836,192,84,0,0,0,0,0,12,2008


### Using the Scaler on a DataFrame

Below, we can compare the results of the two scaling transformations by passing a list of column names to the scaler.  Note the practice of initializing the object, fitting it, and transforming.  

In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [5]:
std_scaled = StandardScaler()
minmax_scaled = MinMaxScaler()

In [6]:
cols = ames_numeric.columns

In [7]:
std_df = std_scaled.fit_transform(ames[[name for name in cols]])
minmax_df = minmax_scaled.fit_transform(ames[[name for name in cols]])

In [8]:
pd.DataFrame(std_df).head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,-1.730865,0.073375,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.575425,-0.288653,-0.944591,...,0.351000,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,0.138777
1,-1.728492,-0.872563,-0.091886,-0.071836,2.179628,0.156734,-0.429577,1.171992,-0.288653,-0.641228,...,-0.060731,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-0.489110,-0.614439
2,-1.726120,0.073375,0.073480,0.651479,-0.517200,0.984752,0.830215,0.092907,-0.288653,-0.301643,...,0.631726,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.990891,0.138777
3,-1.723747,0.309859,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.499274,-0.288653,-0.061670,...,0.790804,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,-1.367655
4,-1.721374,0.073375,0.375148,1.374795,-0.517200,0.951632,0.733308,0.463568,-0.288653,-0.174865,...,1.698485,0.780197,0.563760,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,2.100892,0.138777


In [9]:
pd.DataFrame(minmax_df).head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0.000000,0.235294,0.033420,0.666667,0.500,0.949275,0.883333,0.125089,0.0,0.064212,...,0.386460,0.000000,0.111517,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50
1,0.000685,0.000000,0.038795,0.555556,0.875,0.753623,0.433333,0.173281,0.0,0.121575,...,0.324401,0.347725,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25
2,0.001371,0.235294,0.046507,0.666667,0.500,0.934783,0.866667,0.086109,0.0,0.185788,...,0.428773,0.000000,0.076782,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50
3,0.002056,0.294118,0.038561,0.666667,0.500,0.311594,0.333333,0.038271,0.0,0.231164,...,0.452750,0.000000,0.063985,0.492754,0.0,0.0,0.0,0.0,0.090909,0.00
4,0.002742,0.235294,0.060576,0.777778,0.500,0.927536,0.833333,0.116052,0.0,0.209760,...,0.589563,0.224037,0.153565,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50


### Fit a Linear Model on Scaled Data

In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
lm = LinearRegression()

In [12]:
y = np.log(y)

In [13]:
ames_numeric_scaled = std_scaled.fit_transform(ames[[name for name in cols]])

In [14]:
lm.fit(ames_numeric_scaled, y)

/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
predictions = lm.predict(ames_numeric_scaled)

In [17]:
mse = mean_squared_error(y, predictions)

In [18]:
rmse = np.sqrt(mse)
score = lm.score(ames_numeric_scaled, predictions)

In [19]:
print('R-squared score: {}'.format(score), '\nRMSE: {:.4f}'.format(rmse))

R-squared score: 1.0 
RMSE: 0.1457


### Splitting the Data 

As we have seen, we will tend to overfit the data if we use the entire dataset to determine the model.  To account for this, we will split our datasets into a **training set** to build our model on, and a **test set** to evaluate the performance of the model.  We have a handy sklearn method for doing this, who by default splits the data into 80% for training and 20% for testing.

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(ames_numeric_scaled, y)

In [22]:
lm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [23]:
pred = lm.predict(X_test)

In [24]:
mse = mean_squared_error(y_test, pred)

In [25]:
rmse = np.sqrt(mse)
rmse

0.13108075406894865

### Regularized Methods Comparison



In [26]:
crime = pd.read_csv('data/crime_data.csv', index_col = 'Unnamed: 0')

FileNotFoundError: File b'data/crime_data.csv' does not exist

In [ ]:
crime.head()

In [ ]:
y = crime['ViolentCrimesPerPop']

In [ ]:
X = crime.drop('ViolentCrimesPerPop', axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
lm = LinearRegression()
lm.fit(X_train_scaled, y_train)
predictions = lm.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
score = lm.score(X_test_scaled, y_test)
print('The r2 value is : {:.4f}'.format(score), '\nThe RMSE value is {:.4f}'.format(rmse))

### Ridge Regression

$$RSS(w, b) = \sum_{i = 1} ^ N (y_i - (wx_i + b))^2 + \alpha \sum_{j = 1}^p w_j^2 $$

Many feature coefficients will be determined with small values.  Larger $\alpha$ means larger penalty, zero is base LinearRegression, and the default for sklearn's implementation is 1.0.

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge_reg = Ridge(alpha = 1)

In [ ]:
ridge_reg.fit(X_train_scaled, y_train)

In [ ]:
rpred = ridge_reg.predict(X_test_scaled)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, rpred))
score = ridge_reg.score(X_test_scaled, y_test)
print('The r2 value is : {:.4f}'.format(score), '\nThe RMSE value is {:.4f}'.format(rmse))

In [ ]:
np.sum(ridge_reg.coef_ != 0)

In [ ]:
crime.shape

In [ ]:
ridge_reg = Ridge(alpha = 20)
ridge_reg.fit(X_train_scaled, y_train)
rpred = ridge_reg.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, rpred))
score = ridge_reg.score(X_test_scaled, y_test)
print('The r2 value is : {:.4f}'.format(score), '\nThe RMSE value is {:.4f}'.format(rmse))

### Lasso Regression

$$RSS(w, b) = \sum_{i = 1} ^ N (y_i - (wx_i + b))^2 + \alpha \sum_{j = 1}^p |w_j| $$

Now, we end up in effect setting variables with low influence to a coefficient of zero.  Compared to Ridge, we would use Lasso if there are only a few variables with substantial effects.

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso_reg = Lasso(alpha = 2.0)

In [ ]:
lasso_reg.fit(X_train_scaled, y_train)

In [ ]:
lpred = lasso_reg.predict(X_test_scaled)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, lpred))
score = ridge_reg.score(X_test_scaled, y_test)
print('The r2 value is : {:.4f}'.format(score), '\nThe RMSE value is {:.4f}'.format(rmse))

In [ ]:
np.sum(lasso_reg.coef_ != 0)

In [ ]:
for e in sorted (list(zip(list(X), lasso_reg.coef_)),
                key = lambda e: -abs(e[1])):
    if e[1] != 0:
        print('\t{}, {:.3f}'.format(e[0], e[1]))

### Elastic Net

$$RSS(w, b) = \sum_{i = 1} ^ N (y_i - (wx_i + b))^2 + r\alpha\sum_{i = 1}^n |w_j| + \frac{1-r}{2} \alpha \sum_{j = 1}^p w_j^2 $$



In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
elastic_reg = ElasticNet(alpha = .05, l1_ratio=0.4)
elastic_reg.fit(X_train_scaled, y_train)
epred = elastic_reg.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, epred))
rmse

In [ ]:
ridge_score = ridge_reg.score(X_test_scaled, y_test)
lasso_score = lasso_reg.score(X_test_scaled, y_test)
elastic_score = elastic_reg.score(X_test_scaled, y_test)

In [ ]:
print("Ridge: {:.4f}".format(ridge_score), "\nLasso: {:.4f}".format(lasso_score),
      "\nElastic Net: {:.4f}".format(elastic_score))

In [ ]:
pd.DataFrame(scaled, columns = cols)

### PROBLEM

Return to your Ames Data.  We have covered a lot of ground today, so let's summarize the things we could do to improve the performance of our original model that compared the Above Ground Living Area to the Logarithm of the Sale Price.
<div class="alert alert-info" role="alert">
1. Clean data, drop missing values
2. Transform data, code variables using either ordinal values or OneHotEncoder methods
3. Create more features from existing features
4. Split our data into testing and training sets
5. Normalize quantitative features
6. Use Regularized Regression methods and Polynomial regression to improve performance of model
</div>
Can you use some or all of these ideas to improve upon your initial model?

### Additional Resources

The last two lessons have pulled heavily from these resources.  I recommend them all strongly as excellent resources:

- SciKitLearn documentation on Regression: http://scikit-learn.org/stable/supervised_learning.html#supervised-learning

- Aurelien Geron, *Hands on Machine Learning with SciKitLearn and TensorFlow*

- James et. al, *An Introduction to Statistical Learning: With Applications in R*

- Philipp K. Janert, *Data Analysis with OpenSource Tools*

- University of Michigan Coursera Class on Machine Learning with SciKitLearn: https://www.coursera.org/learn/python-machine-learning

- Stanford University course on Machine Learning: https://www.coursera.org/learn/machine-learning